In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calculateEpsilons(eigenVals, eigenVects, M): # only works for 2 agents
    eps_n = np.sqrt(M) * np.sum((np.abs(eigenVals[1:]) / (1 - np.abs(eigenVals[1:]))))
    vpos = 0
    vneg = 0
    b = 0
    eps_c = 0
    for i in range(M):
        if((eigenVects * np.transpose(eigenVects))[i,i] >= 0):
            vpos = np.sum(eigenVects[i,0] * eigenVects[i,1])
        if((eigenVects * np.transpose(eigenVects))[i,i] <= 0):
            vneg = np.sum(eigenVects[i,0] * eigenVects[i,1])
    
        if(eigenVals[0] * eigenVals[1] >= 0 and (eigenVects * np.transpose(eigenVects))[i,i] >= 0):
            b = vpos * (eigenVects * (np.transpose(eigenVects)))[i,i]
        elif(eigenVals[0] * eigenVals[1] >= 0 and (eigenVects * np.transpose(eigenVects))[i,i] <= 0):
            b = vneg * (eigenVects * (np.transpose(eigenVects)))[i,i]
        else:
            b = max(np.abs(vneg), vpos) * np.abs((eigenVects * np.transpose(eigenVects))[i,i])
    
    for p in range(M - 1):
        lambdas = np.abs(eigenVals[p] * eigenVals[p+1])
        eps_c += (lambdas / (1 - lambdas)) * b
    eps_c *= M
    return eps_n, eps_c

In [3]:
class Agent:
    def __init__(self, num_arms):
        # Number of total steps
        self.total_arm_selections = np.ones(num_arms)
        # Total reward for each arm
        self.arm_rewards = np.zeros(num_arms)
        # Total mean reward from all arms
        self.mean_rewards = self.arm_rewards / self.total_arm_selections
    
    def updateSelections(self, index):
        self.total_arm_selections[index] += 1
    
    def updateRewards(self, index, reward):
        self.arm_rewards[index] = reward
    
    def updateMeans(self):
        self.mean_rewards = self.arm_rewards / self.total_arm_selections

In [4]:
class Arm:
    def __init__(self):
        self.reward_distrib = np.random.normal(0, 1)
    
    def getReward(self):
        return np.random.normal(self.reward_distrib, 1)

In [186]:
N = 5 # number of arms
M = 2 # number of agents
arms = []
agents = []
arm_selections = []
arm_rewards = []
mean_rewards = []

indicators = np.zeros([M,N])
rewards = np.zeros([M,N])

T = 1000 # time to run for
gamma = 1.2 # must be greater than 1 according to paper

for i in range(N): # initialize arms
    arms.append(Arm())

for i in range(M): # initialize agents
    agents.append(Agent(N))

for i in range(len(agents)): # get arrays of selections and rewards
    arm_selections.append(agents[i].total_arm_selections)
    arm_rewards.append(agents[i].arm_rewards)
arm_selections = np.array(arm_selections)
arm_rewards = np.array(arm_rewards)

for i in range(len(arms)): # initialize with 1 round of rewards
    for j in range(len(agents)):
        reward = arms[i].getReward()
        arm_rewards[j][i] = reward
mean_rewards = arm_rewards/arm_selections

In [187]:
G = nx.Graph()
G.add_nodes_from([0, 1])
G.add_edges_from([(0, 1)])
A = nx.adjacency_matrix(G)
L = nx.laplacian_matrix(G)

M = len(G)

maxDeg = 0
for i in range(len(G)): # find max degree in graph
    if G.degree[i] > maxDeg:
        maxDeg = G.degree[i]

kappa = .2
P = np.eye(len(G)) - (kappa*L / maxDeg) # calculate P matrix from paper
eigenVals, eigenVects = np.linalg.eig(P) # calculate P's eigenvectors and eigenvalues

_, eps_c = calculateEpsilons(eigenVals, eigenVects, M) # calculate epsilons eps_n and eps_c from paper

In [188]:
reward = 0
sampledIndex = 0
maxRwd = 0
maxIndex = 0
for t in range(1,T+1):
    for k in range(len(agents)):
        mean_rewards = arm_rewards/arm_selections # update u_i(t)
        arm_selections = P*arm_selections + P*indicators # update s_i(t)
        arm_rewards = P*arm_rewards + P*rewards # update n_i(t)
        
        maxRwd = 0
        for i in range(len(arms)): # loop through arms to find max reward to decide which arm to choose
            val = mean_rewards[k,i] + 1 * np.sqrt((2 * gamma) * 
                        ((arm_selections[k,i] + eps_c) / M*arm_selections[k,i])
                        * (np.log(t) / arm_selections[k,i]))
            if val > maxRwd:
                maxRwd = val
                maxIndex = i
        indicators = np.zeros([M,N])
        rewards = np.zeros([M,N])
        sampledIndex = maxIndex # set index of chosen arm
        reward = arms[sampledIndex].getReward() # get reward from chosen arm
        indicators[k,sampledIndex] = 1 # set indicator to 1 for chosen arm
        rewards[k,sampledIndex] = reward # set r_i(t)

In [189]:
mean_rewards

matrix([[ 0.50048675,  2.18771725, -1.54392025,  0.38185153, -0.29738432],
        [ 0.50048675,  2.1881889 , -1.54392025,  0.38185153, -0.29738432]])

In [190]:
arm_selections

matrix([[1.5000000e+00, 1.0001875e+03, 1.0000000e+00, 1.0000000e+00,
         1.0000000e+00],
        [1.5000000e+00, 9.9981250e+02, 1.0000000e+00, 1.0000000e+00,
         1.0000000e+00]])

In [191]:
arm_rewards

matrix([[ 7.50730128e-01,  2.18725993e+03, -1.54392025e+00,
          3.81851531e-01, -2.97384324e-01],
        [ 7.50730128e-01,  2.18744380e+03, -1.54392025e+00,
          3.81851531e-01, -2.97384324e-01]])